In [10]:
import yfinance as yf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats as si
from mpl_toolkits.mplot3d import Axes3D

In [ ]:
tesla_data = yf.download('TSLA', start='2022-04-11', end='2024-04-11')

In [ ]:
plt.figure(figsize=(14, 7))
plt.plot(tesla_data['Close'], label='Tesla Closing Price')
plt.title('Tesla Equity Price Movement Over the Last Two Years')
plt.xlabel('Date')
plt.ylabel('Price (USD)')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
daily_returns = tesla_data['Close'].pct_change().dropna()
average_daily_return = daily_returns.mean()
annualized_avg_return = (1 + average_daily_return) ** 252 - 1  # 252 trading days in a year

annualized_std_dev = daily_returns.std() * np.sqrt(252)

print(f"Annualized Average Return: {annualized_avg_return}")
print(f"Annualized Standard Deviation: {annualized_std_dev}")

In [ ]:
tesla_data.describe().round(2)

In [ ]:
S0 = 800  # Initial stock price, e.g., Tesla's current stock price
K = 850   # Strike price
T = 1     # Time to maturity in years
r = 0.02  # Risk-free rate
sigma = 0.48  # Volatility (annualized standard deviation)
N = 3  # Number of steps in the binomial treeÜ
payoff = "put"          # payoff 

In [ ]:
dT = float(T) / N                             # Delta t
u = np.exp(sig * np.sqrt(dT))                 # up factor
d = 1.0 / u   

In [ ]:
S = np.zeros((N + 1, N + 1))
S[0, 0] = S0
z = 1
for t in range(1, N + 1): #looping forwards, from 1 to N
    for i in range(z):  #looping forwards, from 0 to z-1
        S[i, t] = S[i, t-1] * u
        S[i+1, t] = S[i, t-1] * d
    z += 1  # same as z=z+1

In [ ]:
S

In [ ]:
a = np.exp(r * dT)    # risk free compound return
p = (a - d)/ (u - d)  # risk neutral up probability
q = 1.0 - p           # risk neutral down probability
p

In [ ]:
S_T = S[:,-1]
V = np.zeros((N + 1, N + 1))
if payoff =="call":
    V[:,-1] = np.maximum(S_T-K, 0.0)
elif payoff =="put":
    V[:,-1] = np.maximum(K-S_T, 0.0)
V

In [ ]:
# for European Option
for j in range(N-1, -1, -1): # Column. looping backwards. From N-1 to 0
    for i in range(j+1):  # Row. looping forwards. From 0 to j
        V[i,j] = np.exp(-r*dT) * (p * V[i,j + 1] + q * V[i + 1,j + 1]) #the theoretical value at each node.
V

In [ ]:
print('European ' + payoff, str( V[0,0]))

In [ ]:
def mcs_simulation_np(p):
    M = p
    I = p
    dt = T / M 
    S = np.zeros((M + 1, I))
    S[0] = S0 
    rn = np.random.standard_normal(S.shape) 
    for t in range(1, M + 1): 
        S[t] = S[t-1] * np.exp((r - sigma ** 2 / 2) * dt + sigma * np.sqrt(dt) * rn[t])    
        # Price process, see Hilpisch (2015) chapter 1 (equation 1-1) & chapter 3 (equation 3-6)
    return S

In [ ]:
S0 = 800               # spot stock price
K = 850               # strike
T = 1.0                 # maturity 
r = 0.02                 # risk free rate 
sigma = 0.48 

In [ ]:
S = mcs_simulation_np(1000)

In [ ]:
S = np.transpose(S)
S

In [ ]:
import matplotlib.pyplot as plt
n, bins, patches = plt.hist(x=S[:,-1], bins='auto', color='#0504aa',alpha=0.7, rwidth=0.85)

plt.grid(axis='y', alpha=0.75)
plt.xlabel('S_T')
plt.ylabel('Frequency')
plt.title('Frequency distribution of the simulated end-of-preiod values')

In [ ]:
p = np.exp(-r*T)*np.mean(np.maximum(K - S[:,-1],0))
print('European put', str(p))

In [ ]:
c = np.exp(-r*T)*np.mean(np.maximum(S[:,-1] - K,0))
print('European call', str(c))

In [ ]:
def delta(S, K, T, r, vol, payoff):
    
    d1 = (np.log(S / K) + (r + 0.5 * vol ** 2) * T) / (vol * np.sqrt(T))
    d2 = (np.log(S / K) + (r - 0.5 * vol ** 2) * T) / (vol * np.sqrt(T))
    if payoff == "call":
        delta = si.norm.cdf(d1, 0.0, 1.0)
    elif payoff == "put":
        delta = si.norm.cdf(d1, 0.0, 1.0) - 1
    
    return delta

In [ ]:
S = np.linspace(600, 1000, 400)
T = np.linspace(0.5, 2, 51)
Delta = np.zeros((len(T),len(S)))
for j in range(len(S)):
    for i in range(len(T)):
        Delta[i,j] = delta(S[j], 850, T[i], 0.02, 0.48, 'call')

In [ ]:
fig = plt.figure(figsize=(10, 6))
ax = fig.add_subplot(projection = '3d')
S, T = np.meshgrid(S, T)
surf = ax.plot_surface(S, T, Delta, rstride=2, cstride=2, cmap=plt.cm.coolwarm, linewidth=0.5, antialiased=True)
#rstride: the array of row stride (step size) cstride: the array of column stride.
ax.set_xlabel('Stock Price')
ax.set_ylabel('Time to Expiry')
ax.set_zlabel('Delta')
fig.colorbar(surf, shrink=0.5, aspect=5);

In [ ]:
d = delta(800, 850, 1, 0.02, 0.48, 'call')
print('The value of Delta is', d.round(4),'.','If the stock price increase 1 dollar, then the value of the option will increase $', d.round(4), '.')

In [ ]:
def gamma(S, K, T, r,  vol, payoff):
    
    d1 = (np.log(S / K) + (r  + 0.5 * vol ** 2) * T) / (vol * np.sqrt(T))

    gamma = si.norm.pdf(d1, 0.0, 1.0) / (vol *  np.sqrt(T) * S)

    
    
    return gamma

In [ ]:
S = np.linspace(600, 1000, 400)
T = np.linspace(0.5, 2, 51)
Gamma = np.zeros((len(T),len(S)))
for j in range(len(S)):
    for i in range(len(T)):
        Gamma[i,j] = gamma(S[j], 850, T[i], 0.02, 0.48, 'call')

In [ ]:
fig = plt.figure(figsize=(10, 6))
ax = fig.add_subplot(projection = '3d')
S, T = np.meshgrid(S, T)
surf = ax.plot_surface(S, T, Gamma, rstride=2, cstride=2, cmap=plt.cm.coolwarm, linewidth=0.5, antialiased=True)
ax.set_xlabel('Stock Price')
ax.set_ylabel('Time to Expiry')
ax.set_zlabel('Gamma')
fig.colorbar(surf, shrink=0.5, aspect=5);

In [ ]:
g = gamma(800, 850, 1, 0.02, 0.48, 'call')
print('The value of Gamma is', g.round(4),'.','If the volatility increases 1%, then the value of the option will increase $', g.round(4)*0.01, '.')

In [ ]:
def theta(S, K, T, r, vol, payoff):
    
    d1 = (np.log(S / K) + (r + 0.5 * vol ** 2) * T) / (vol * np.sqrt(T))
    d2 = (np.log(S / K) + (r - 0.5 * vol ** 2) * T) / (vol * np.sqrt(T))
    N_d1_prime=1/np.sqrt(2 * np.pi) * np.exp(-d1**2/2)
    
    if payoff == "call":
        theta = - S * N_d1_prime * vol / (2 * np.sqrt(T)) - r * K * np.exp(-r * T) * si.norm.cdf(d2, 0.0, 1.0)
    elif payoff == "put":
        theta = - S * N_d1_prime * vol / (2 * np.sqrt(T)) + r * K * np.exp(-r * T) * si.norm.cdf(-d2, 0.0, 1.0)
    
    return theta

In [ ]:
S = np.linspace(600, 1000, 400)
T = np.linspace(0.5, 2, 51)
Theta = np.zeros((len(T),len(S)))
for j in range(len(S)):
    for i in range(len(T)):
        Theta[i,j] = theta(S[j], 850, T[i], 0.02, 0.48, 'call')

In [ ]:
fig = plt.figure(figsize=(10, 6))
ax = fig.add_subplot(projection = '3d')
S, T = np.meshgrid(S, T)
surf = ax.plot_surface(S, T, Theta, rstride=2, cstride=2, cmap=plt.cm.coolwarm, linewidth=0.5, antialiased=True)
ax.set_xlabel('Stock Price')
ax.set_ylabel('Time to Expiry')
ax.set_zlabel('Theta')
fig.colorbar(surf, shrink=0.5, aspect=5);

In [ ]:
t = theta(800, 850, 1, 0.02, 0.48, 'call')
print('The value of Theta is', t.round(4),'.','If the volatility increases 1%, then the value of the option will increase $', t.round(4)*0.01, '.')

In [ ]:
def vega(S, K, T, r, vol, payoff):
    
    d1 = (np.log(S / K) + (r + 0.5 * vol ** 2) * T) / (vol * np.sqrt(T))
    N_d1_prime=1/np.sqrt(2 * np.pi) * np.exp(-d1**2/2)
    vega = S * np.sqrt(T) * N_d1_prime
    
    return vega

In [ ]:
S = np.linspace(600, 1000, 400)
T = np.linspace(0.5, 2, 51)
Vega = np.zeros((len(T),len(S)))
for j in range(len(S)):
    for i in range(len(T)):
        Vega[i,j] = vega(S[j], 850, T[i], 0.02, 0.48, 'call')

In [ ]:
fig = plt.figure(figsize=(10, 6))
ax = fig.add_subplot(projection = '3d')
S, T = np.meshgrid(S, T)
surf = ax.plot_surface(S, T, Vega, rstride=2, cstride=2, cmap=plt.cm.coolwarm, linewidth=0.5, antialiased=True)
ax.set_xlabel('Stock Price')
ax.set_ylabel('Time to Expiry')
ax.set_zlabel('Vega')
fig.colorbar(surf, shrink=0.5, aspect=5);

In [ ]:
v = vega(800, 850, 1, 0.02, 0.48, 'call')
print('The value of Vega is', v.round(4),'.','If the volatility increases 1%, then the value of the option will increase $', v.round(4)*0.01, '.')

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Parameters
S0 = 800    # Initial stock price
K = 850     # Strike price
T = 1       # Time to expiry
n = 3       # Number of steps in the binomial tree
r = 0.02    # Risk-free interest rate per year
sigma = 0.48 # Volatility of the underlying asset
dt = T / n  # Length of each time step
u = np.exp(sigma * np.sqrt(dt))  # Up factor per time step
d = np.exp(-sigma * np.sqrt(dt))  # Down factor per time step

# Initialize the arrays to store stock prices and option values
stock_prices = np.zeros((n+1, n+1))
option_values = np.zeros((n+1, n+1))

# Generate stock prices in the binomial tree
for i in range(n+1):
    for j in range(i+1):
        stock_prices[j, i] = S0 * (u ** (i - j)) * (d ** j)

# Calculate the option values at expiry (European call option)
option_values[:, n] = np.maximum(stock_prices[:, n] - K, 0)

# Backward induction for option valuation
for i in range(n-1, -1, -1):
    for j in range(i+1):
        option_values[j, i] = np.exp(-r * dt) * (option_values[j, i+1] * u + option_values[j+1, i+1] * d) / (u + d)

# Plotting the binomial tree
fig, ax = plt.subplots(figsize=(10, 6))
for i in range(n):
    for j in range(i+1):
        if j < i+1:
            ax.plot([i, i+1], [stock_prices[j, i], stock_prices[j, i+1]], 'bo-', linewidth=2, markersize=8)
            ax.plot([i, i+1], [stock_prices[j, i], stock_prices[j+1, i+1]], 'bo-', linewidth=2, markersize=8)

# Annotating the final prices and option values
for i in range(n+1):
    for j in range(i+1):
        ax.annotate(f'{stock_prices[j, i]:.2f}', (i, stock_prices[j, i]), textcoords="offset points", xytext=(0,10), ha='center')
        if i == n:
            ax.annotate(f'({option_values[j, i]:.2f})', (i, stock_prices[j, i]), textcoords="offset points", xytext=(0,-15), ha='center')

ax.set_title('European Call Option Pricing: Binomial Tree')
ax.set_xlabel('Time Steps')
ax.set_ylabel('Stock Price and Option Value')
plt.grid(True)
plt.show()


In [ ]:
def delta(S, K, T, r, vol, payoff):
    
    d1 = (np.log(S / K) + (r + 0.5 * vol ** 2) * T) / (vol * np.sqrt(T))
    if payoff == "call":
        delta = si.norm.cdf(d1, 0.0, 1.0)
    elif payoff == "put":
        delta =  si.norm.cdf(d1, 0.0, 1.0)-1
    
    return delta

In [ ]:
S = np.linspace(600, 1000, 400)
Delta_Call = np.zeros((len(S),1))
Delta_Put = np.zeros((len(S),1))
for i in range(len(S)):
    Delta_Call [i] = delta(S[i], 850, 1, 0.02, 0.48, 'call')
    Delta_Put [i] = delta(S[i], 850, 1, 0.02, 0.48, 'put')

In [ ]:
fig = plt.figure()
plt.plot(S, Delta_Call, '-')
plt.plot(S, Delta_Put, '--')
plt.grid()
plt.xlabel('Stock Price')
plt.ylabel('Delta')
plt.title('Delta')
plt.legend(['Delta for Call','Delta for Put'])

In [ ]:
def gamma(S, K, T, r,  vol, payoff):
    
    d1 = (np.log(S / K) + (r  + 0.5 * vol ** 2) * T) / (vol * np.sqrt(T))

    gamma = si.norm.pdf(d1, 0.0, 1.0) / (vol *  np.sqrt(T) * S)

    
    return gamma

In [ ]:
S = np.linspace(600, 1000, 400)
Gamma = np.zeros((len(S),1))
for i in range(len(S)):
    Gamma [i] = gamma(S[i], 850, 1, 0.02, 0.48, 'call')

In [ ]:
fig = plt.figure()
plt.plot(S, Gamma, '-')
plt.grid()
plt.xlabel('Stock Price')
plt.ylabel('Gamma')
plt.title('Gamma')
plt.legend(['Gamma for Call and Put'])

In [ ]:
def theta(S, K, T, r, vol, payoff):
    
    d1 = (np.log(S / K) + (r + 0.5 * vol ** 2) * T) / (vol * np.sqrt(T))
    d2 = (np.log(S / K) + (r - 0.5 * vol ** 2) * T) / (vol * np.sqrt(T))
    N_d1_prime=1/np.sqrt(2 * np.pi) * np.exp(-d1**2/2)
    
    if payoff == "call":
        theta = - S * N_d1_prime * vol / (2 * np.sqrt(T)) - r * K * np.exp(-r * T) * si.norm.cdf(d2, 0.0, 1.0)
    elif payoff == "put":
        theta = - S * N_d1_prime * vol / (2 * np.sqrt(T)) + r * K * np.exp(-r * T) * si.norm.cdf(-d2, 0.0, 1.0)
    
    return theta

In [ ]:
T = np.linspace(0.25,3,12)
Theta_Call = np.zeros((len(T),1))
Theta_Put = np.zeros((len(T),1))
for i in range(len(T)):
    Theta_Call [i] = theta(800, 850, T[i], 0.02, 0.48, 'call')
    Theta_Put [i] = theta(800, 850, T[i], 0.02, 0.48, 'put')

In [ ]:
fig = plt.figure()
plt.plot(T, Theta_Call, '-')
plt.plot(T, Theta_Put, '-')
plt.grid()
plt.xlabel('Time to Expiry')
plt.ylabel('Theta')
plt.title('Theta')
plt.legend(['Theta for Call', 'Theta for Put'])

In [ ]:
def vega(S, K, T, r, vol, payoff):
    
    d1 = (np.log(S / K) + (r + 0.5 * vol ** 2) * T) / (vol * np.sqrt(T))
    N_d1_prime=1/np.sqrt(2 * np.pi) * np.exp(-d1**2/2)
    vega = S * np.sqrt(T) * N_d1_prime
    
    return vega

In [ ]:
vol = np.linspace(0.1,0.7,13)
Vega = np.zeros((len(vol),1))
for i in range(len(vol)):
    Vega [i] = vega(800, 850, 1, 0.02, vol[i], 'call')

In [ ]:
fig = plt.figure()
plt.plot(vol, Vega, '-')
plt.grid()
plt.xlabel('Volatility')
plt.ylabel('Vega')
plt.title('Vega')
plt.legend(['Vega for Call and Put'])